In [6]:
import pandas as pd
import numpy as np

In [9]:
# Extract all csv files from the zipped folders titled dataPart1, dataPart2, dataPart3, and dataPart4

ranges = [pd.read_csv('range1_df1.csv'),
          pd.read_csv('range2_df1.csv'),
          pd.read_csv('range3_df1.csv'),
          pd.read_csv('range4_df1.csv'),
          pd.read_csv('range5_df1.csv'),
          pd.read_csv('range6_df1.csv'),
          pd.read_csv('range7_df1.csv'),
          pd.read_csv('range8_df1.csv')]

all_data_df = pd.concat(ranges)

In [8]:
#'belongs to no denomination but do divine service at home'
#'ha? made no ch? for himself yet'
#'egg dealer'
#'druidical'
#'roscommon'
#'a sinner'

del all_data_df[all_data_df.columns[0]] # Contains meaningless index
del all_data_df[all_data_df.columns[0]] # Contains meaningless index

data = all_data_df.copy()

# Add headings for columns
headings = ["Surname","Forename","Townland/Street","DED","County","Age",
            "Sex","Birthplace","Occupation","Religion","Literacy","Irish Language",
            "Relation to Head of Household","Marital Status","Specified Illnesses",
            "Years Married","Children Born","Children Living"]
data.columns = headings

# Replace old county names with modern names
data['County'] = data['County'].replace(to_replace={'Londonderry':'Derry','King\'s Co.':'Offaly','Queen\'s Co.':'Laois'})

# Make all Sex and Religion values lower case for easier processing
data['Sex'] = data['Sex'].str.lower()
data['Religion'] = data['Religion'].str.lower()

# Many columns devoted to categorizing the strange Religion spellings recorded on the census forms
catholic = 'catherolic|cathiloc|catoiliceach|catholis|sisters of the sacred heart of mary|catlick|catherlic|cathloic|cathelic|rom cath|catherlick|catolique|c church|chatholic|catolic|catcholic|cathorlic|catholique|cathlic|roman|r c|church of rome|roman c|chatolic'
protestant = 'member of irish church|chruch of england|irish ch|ch of e|member church ireland|late established church|e church|c h i|irish curch|anglican|curch of england|ch of i|irish protestant church|established church|ch of eng|curch ireland|church off ireland|church of i|church of eng|ch ireland|ch of england|curch of ireland|english church|ch of ireland|church of england|church ireland|irish church|c of e|c e|protestant|i church|i c|c of i|c i|church england|episcopalian|episcopal|epscoplian|episcopelian|episcoplian|episcopilian|irish episcopal church|epescopalian|episcopolian'
presbyterian = 'pesbyterian|prysbterian|prebesterian|bresbyterian|united free|u f c of scotland|u f church|u f c scotland|prisbeterian|prysbeterian|prisbyterian|covenanter|united free church|prebysterian|presbyterian|presbeterian|prespyterian|united free church of scotland|prespeterian|church of scotland|presby|pres|presbytarian|reformed presbyterian'
methodist = 'methodist|wesleyan'
baptist = 'baptist|babtist'
unitarian = 'unitarian|uniterian|unaterian|unatiran|unatarian|uintarian|unateran|unesterian|wnuterian|unaterain|unatarion|christadelphian'
quaker = 'society of friends|quaker|society of friend'
jewish = 'hebrew|jewish|jew'
christian = 'a christian|christain|christian|church|free church'
otherProtestants = 'salvationist|salvation army|church of christ|lutheran|morarian|movarian|moraian|moravian|moran|moro|morevian|morivan|moreavins|moreraviens|monarian|marevian|plymouth|christian brethren|christian brethern|brethren|brethern|breathren|breathern|bretheren|church of god|mormon|congregationalist|congregationist|congregational|congreational|congregationlist|apostolic church|undenominational christian protestant|seventh day adventist|pilgrim'
noReligion = 'atheist|undenominational|free thinker|freethinker|no religion|independent|agnostic|no denomination|independant|non sectarian'
worldReligions = 'hindu|zorost|zoroastrianism|buddhist|buddist|buddha|islam|mahommed'
unspecified = 'information refused|refused|a|b|c|d|e|f|g|h|i|j|k|l|m|n|o|p|q|r|s|t|u|v|w|x|y|z|1|2|3|4|5|6|7|8|9|0|\(|\?|\-|\"'
data.loc[data['Religion'].str.contains('catholic|rome', na=False), 'Religion'] = 'Catholic'
data.loc[data['Religion'].str.match(catholic, na=False), 'Religion'] = 'Catholic'
data.loc[data['Religion'].str.contains('church of ireland', na=False), 'Religion'] = 'Protestant'
data.loc[data['Religion'].str.match(protestant, na=False), 'Religion'] = 'Protestant'
data.loc[data['Religion'].str.contains('presbyterian|church of scotland', na=False), 'Religion'] = 'Presbyterian'
data.loc[data['Religion'].str.match(presbyterian, na=False), 'Religion'] = 'Presbyterian'
data.loc[data['Religion'].str.contains('methodist|wesleyan', na=False), 'Religion'] = 'Methodist'
data.loc[data['Religion'].str.match(baptist, na=False), 'Religion'] = 'Baptist'
data.loc[data['Religion'].str.match(unitarian, na=False), 'Religion'] = 'Unitarian'
data.loc[data['Religion'].str.match(quaker, na=False), 'Religion'] = 'Quaker'
data.loc[data['Religion'].str.match(jewish, na=False), 'Religion'] = 'Jewish'
data.loc[data['Religion'].str.match(christian, na=False), 'Religion'] = 'Unspecified Christian'
data.loc[data['Religion'].str.contains('brethren|brethern', na=False), 'Religion'] = 'Other Protestants'
data.loc[data['Religion'].str.match(otherProtestants, na=False), 'Religion'] = 'Other Protestants'
data.loc[data['Religion'].str.match(worldReligions, na=False), 'Religion'] = 'World religion'
data.loc[data['Religion'].str.match(noReligion, na=False), 'Religion'] = 'No religion'
data.loc[data['Religion'].str.match(unspecified, na=False), 'Religion'] = 'Unspecified'
prodStrings = 'Protestant contains Church of Ireland, Church of England, and Episcopalian'
otherProdStrings = 'Other Protestant contains Salvationists, Lutherans, Moravians, Christian Brethren, Mormons, Congregationalists, and Adventists'
worldRelString = 'World religions contains Muslims, Hindus, Buddhists, and Zorostrians'

# Cleaning and unifying Occupation data
student = 'Student|Scholar|Scholars|Pupil|Attending School|School Boy'
farmerSon = 'Farmer\'s son|Farmers son|Farmer\'s Son|Farmers Son|Farmer Son'
farmerDaughter = 'Farmer\s daughter|Farmers daughter|Farmers Daughter'
genLabourer = 'General Labourer|Labourer General'
servant = 'Domestic Servant|General Servant Domestic|General Servant|General Domestic Servant|Domestic'
dressmaker = 'Dress Maker|Dressmaker'
housekeeper = 'House Keeper|Housekeeper'
farmLabourer = 'Farm Labourer'
labourer = 'General labourer'
teacher = 'National Teacher'
data.loc[data['Occupation'].str.match(student, na=False), 'Occupation'] = 'Student'
data.loc[data['Occupation'].str.match(farmerSon, na=False), 'Occupation'] = 'Farmer\'s Son'
data.loc[data['Occupation'].str.match(farmerDaughter, na=False), 'Occupation'] = 'Farmer\'s daughter'
data.loc[data['Occupation'].str.contains(genLabourer, na=False), 'Occupation'] = 'General labourer'
data.loc[data['Occupation'].str.contains(servant, na=False), 'Occupation'] = 'Servant'
data.loc[data['Occupation'].str.contains(dressmaker, na=False), 'Occupation'] = 'Dressmaker'
data.loc[data['Occupation'].str.contains(housekeeper, na=False), 'Occupation'] = 'Housekeeper'
data.loc[data['Occupation'].str.contains(farmLabourer, na=False), 'Occupation'] = 'Agricultural Labourer'
data.loc[data['Occupation'].str.contains(labourer, na=False), 'Occupation'] = 'Labourer'
data.loc[data['Occupation'].str.contains(teacher, na=False), 'Occupation'] = 'National School Teacher'
data.loc[data['Occupation'].str.contains('Black Smith', na=False), 'Occupation'] = 'Blacksmith'
data.loc[(data['Occupation'] == 'Student') & (data['Age'] < 13), 'Occupation'] = 'Primary School Student'
data.loc[(data['Occupation'] == 'Student') & (data['Age'].between(13, 17, inclusive=True)), 'Occupation'] = 'Secondary School Student'
data.loc[(data['Occupation'] == 'Student') & (data['Age'] > 17), 'Occupation'] = 'University Student'

# Cleaning the birthplace data
data['Birthplace'] = data['Birthplace'].str.lower()
for i in ['cork','antrim','down','mayo','galway','dublin','donegal','kerry','tipperary','tyrone','armagh',\
         'limerick','wexford','clare','cavan','roscommon','derry','sligo','kilkenny','monaghan','meath','laois',\
         'leitrim','waterford','wicklow','louth','fermanagh','kildare','longford','westmeath','carlow','offaly']:
    data.loc[data['Birthplace'].str.contains(i, na=False), 'Birthplace'] = i
data.loc[data['Birthplace'].str.contains('armagh', na=False), 'Birthplace'] = 'armagh'
data.loc[data['Birthplace'].str.contains('belfast', na=False), 'Birthplace'] = 'antrim'
data.loc[data['Birthplace'].str.contains('king', na=False), 'Birthplace'] = 'offaly'
data.loc[data['Birthplace'].str.contains('queen', na=False), 'Birthplace'] = 'laois'
data.loc[data['Birthplace'].str.contains('tipp', na=False), 'Birthplace'] = 'tipperary'
data.loc[data['Birthplace'].str.contains('newry', na=False), 'Birthplace'] = 'down'
data.loc[data['Birthplace'].str.contains('limk', na=False), 'Birthplace'] = 'limerick'
data.loc[data['Birthplace'].str.contains('glasgow', na=False), 'Birthplace'] = 'scotland'
data.loc[data['Birthplace'].str.contains('liverpool|london|manchester|birmingham', na=False), 'Birthplace'] = 'england'

# Irish language entries
data['Irish Language'] = data['Irish Language'].str.lower()
bilingual = 'irish and english|english and irish|irish english|irish and eng|english irish|irish & english|irish and e|i and e|lrish and english|irish eng|e and irish|irish|irish and engh|i and english|eng and irish|irish and engl|irish and englishlish|i and eng'
irish = 'irish only'
english = 'english|english only|english language|eng|no|only english|inglish|no irish'
data.loc[data['Irish Language'].str.fullmatch(bilingual, na=False), 'Irish Language'] = 'English & Irish'
data.loc[data['Irish Language'].str.fullmatch(irish, na=False), 'Irish Language'] = 'Irish only'
data.loc[data['Irish Language'].str.fullmatch(english, na=False), 'Irish Language'] = 'English only'

# Literacy entries
data['Literacy'] = data['Literacy'].str.lower()
rw = 'read and write|read write|can read and write|read and rite|yes|r and w|both|write and read|can read write|scholar|read and wright|read rite|read and right|r and write|can read or write|both read and write|write read|write|read an write|read wright|read or write|yes read and write|can read and rite|read right|read and w|scholars|r write'
readOnly = 'read|read only|can read|can read only|cannot write|cannot read write|cannot r or w|canot read'
neither = 'cannot read|cannot read or write|cannot|cannot read and write|cant read or write|no|neither|cannot read nor write|cant read|cant read and write|neither read or write|can\'t read'
data.loc[data['Literacy'].str.fullmatch(rw, na=False), 'Literacy'] = 'Literate'
data.loc[data['Literacy'].str.fullmatch(readOnly, na=False), 'Literacy'] = 'Can only read'
data.loc[data['Literacy'].str.fullmatch(neither, na=False), 'Literacy'] = 'Cannot read or write'

In [3]:
data.to_csv('cleanData.csv')